## Question 4: We are tasked to classify text into a given category(1,0). We choose `decision tree` and `naive bayes` as our classifiers

The plan to do so is:
1. Put the text data into a dataframe
2. Clean the text data by removing stop words, numbers, punctuation, and by lemmatizing each word, as well as converting it to lower case
3. Convert each observation into a vector which is of length n where n is the amount of unique words in our dataset. Each element of the vector will correspond to each unique word in the dataset, showing the frequency of said word in the given observation. 
4. Split the dataset into train and test (80:20)
5. Test on each classifier
6. Calculate the performance of each classifier with accuracy, precision, recall, and F1
7. Time permitting, also try with step 3 replaced by word embeddings

In [48]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

Step 1

In [34]:
data = pd.read_csv("Data/musical.tsv", sep="\t")

Step 2

In [35]:
en_stops = set(stopwords.words('english'))
data["Review"] = data["Review"].\
    apply(lambda x : ' '.join([word.lower() for word in x.split() if word not in (en_stops)]))
data["Review"] = data["Review"].str.replace('[^a-z\s]', '')

/tmp/ipykernel_104564/819155829.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data["Review"] = data["Review"].str.replace('[^a-z\s]', '')


In [36]:
data

,Review,Score
0,this second set strap locks ive owned they lit...,1
1,first i want say i love tube amp distortion ov...,1
2,bought idea full version behringers sequence p...,0
3,if like me probably bought hook xlr microphone...,1
4,didnt know expect proved worth gamblethis cab...,1
...,...,...
995,it really pains give anything star review boss...,1
996,its decent unit stopped working completely mo...,0
997,i bought cable order able run longer cable run...,1
998,well made works should however seem getting li...,1


Step 3

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

docs = data["Review"]
vec = CountVectorizer()
bow = vec.fit_transform(data["Review"])
bow = pd.DataFrame(bow.toarray(), columns=vec.get_feature_names())

/home/monty/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [47]:
processed_data = pd.concat([data["Score"], bow], axis=1)

In [53]:
X = processed_data.loc[:, processed_data.columns != "Score" ]
y = processed_data["Score"]